In [6]:
#accessing netcdf files from Hector's folders
import os, glob, sys
sys.path.append("//nobackup//amondal//Python//Hector_Python_Scripts")
sys.path.append("//nobackup/amondal//Python//mit_geos_analysis")
#playing with netcdf - xarray
import numpy as np
import xarray as xr
import dask.array as da
import matplotlib.pyplot as plt
import dask_ndfilters
# How about visualization?
%matplotlib inline
from xmovie import Movie
import xrscipy.signal as dsp
import xrft
import scipy.signal as scp
import netCDF4
import xgcm
from netCDF4 import Dataset
import time as tm
from xmitgcm import open_mdsdataset
import time as tm
import pylab as plt
from llcmap_bi_split import LLCMap_bi_split
from face_connections import face_connections
from llcmap_nea_split import LLCMap_nea_split
from timeline_MITgcm import timeline
from datetime import datetime, timedelta
import importlib
from GetNCDataSet import getMITNCDataSet
from MITllc2160Depth import *
#GetNCDataSet = importlib.reload(GetNCDataSet)

from dask.distributed import Client
client = Client(memory_limit='30GB',n_workers = 10, threads_per_worker=1)

I'm going to use this notebook to do my first analysis of GEOS data and see what variables I'm going to need from it to calculate spectral budgets/fluxes. This will help with using Hector's GEOS_xr.py code as well as with just doing the calculations for what everything should look like. 

#accessing netcdf files from Hector's folders
import os, glob, sys
#playing with netcdf - xarray
import numpy as np
import xarray as xr
import dask.array as da
import dask_ndfilters

%matplotlib inline
import holoviews as hv
from holoviews.operation.datashader import regrid
hv.extension('bokeh')
import xmovie
from xmovie import Movie
from IPython.display import Video

In [7]:
#Let me open up one of the netcdf4 files that are in Dimitris' directories:
diro = "/nobackupp11/dmenemen/DYAMOND/c1440_llc2160/holding/geosgcm_surf/"
file = "DYAMOND_c1440_llc2160.geosgcm_surf.20200429_0830z.nc4"
gsamp = xr.open_dataset(diro+file)

In [4]:
gsamp

<xarray.Dataset>
Dimensions:       (lat: 8640, lon: 1440, time: 1)
Coordinates:
  * lon           (lon) float64 305.0 305.1 305.1 305.2 ... 34.87 34.92 34.97
  * lat           (lat) float64 -35.25 -35.2 -35.15 ... -35.34 -35.31 -35.29
  * time          (time) datetime64[ns] 2020-04-29T08:30:00
Data variables: (12/64)
    ALBNF         (time, lat, lon) float32 ...
    ALBNR         (time, lat, lon) float32 ...
    ALBVF         (time, lat, lon) float32 ...
    ALBVR         (time, lat, lon) float32 ...
    ASNOW         (time, lat, lon) float32 ...
    BASEFLOW      (time, lat, lon) float32 ...
    ...            ...
    VS            (time, lat, lon) float32 ...
    WET1          (time, lat, lon) float32 ...
    WET2          (time, lat, lon) float32 ...
    WET3          (time, lat, lon) float32 ...
    Z0            (time, lat, lon) float32 ...
    Z0H           (time, lat, lon) float32 ...
Attributes:
    History:      File written by MAPL_CFIO
    Source:       Heracles-5_3
    Title:        DYAMOND_c1440_llc2160
    Contact:      http://gmao.gsfc.nasa.gov
    Conventions:  COARDS
    Institution:  NASA Global Modeling and Assimilation Office
    References:   http://gmao.gsfc.nasa.gov
    Comment:      DYAMOND_c1440_llc2160

I suspect the quantities you're looking for are going to be in geosgcm_turb -- hector mentioned that these are fluxes! geosgcm_tend - contains derivatives -- and geosgcm_surf includes "surface" quantities. It would be good to understand how they're defining surface and also to check where this sponge layer is accounted for in all this data -- (or is it accounted for at all, is it just a backend computational trick?)

In [8]:
#Let me open up one of the netcdf4 files that are in Dimitris' directories:
diro = "/nobackupp11/dmenemen/DYAMOND/c1440_llc2160/holding/geosgcm_tend/"
file = "DYAMOND_c1440_llc2160.geosgcm_tend.20200301_0900z.nc4"
tendsamp = xr.open_dataset(diro+file)

In [9]:
tendsamp

<xarray.Dataset>
Dimensions:   (lat: 8640, lev: 52, lon: 1440, time: 1)
Coordinates:
  * lon       (lon) float64 305.0 305.1 305.1 305.2 ... 34.82 34.87 34.92 34.97
  * lat       (lat) float64 -35.25 -35.2 -35.15 -35.11 ... -35.34 -35.31 -35.29
  * lev       (lev) float64 21.0 22.0 23.0 24.0 25.0 ... 69.0 70.0 71.0 72.0
  * time      (time) datetime64[ns] 2020-03-01T09:00:00
Data variables: (12/20)
    DPDTPHY   (time, lev, lat, lon) float32 ...
    DQVDTCHM  (time, lev, lat, lon) float32 ...
    DQVDTDYN  (time, lev, lat, lon) float32 ...
    DQVDTMST  (time, lev, lat, lon) float32 ...
    DQVDTTRB  (time, lev, lat, lon) float32 ...
    DTDTDYN   (time, lev, lat, lon) float32 ...
    ...        ...
    DUDTMST   (time, lev, lat, lon) float32 ...
    DUDTTRB   (time, lev, lat, lon) float32 ...
    DVDTDYN   (time, lev, lat, lon) float32 ...
    DVDTGWD   (time, lev, lat, lon) float32 ...
    DVDTMST   (time, lev, lat, lon) float32 ...
    DVDTTRB   (time, lev, lat, lon) float32 ...
Attributes:
    History:      File written by MAPL_CFIO
    Source:       Heracles-5_3
    Title:        DYAMOND_c1440_llc2160
    Contact:      http://gmao.gsfc.nasa.gov
    Conventions:  COARDS
    Institution:  NASA Global Modeling and Assimilation Office
    References:   http://gmao.gsfc.nasa.gov
    Comment:      DYAMOND_c1440_llc2160